# Test Case LVV-T1600
This case will verify that the integration of the Camera Hexapod with SAL.
The blocks below represent the steps of the test case.

Requirements
* EFD
* Hexapod(s)powered on
* Thermal sensors attached to the six actuators of the hexapod.
* CSC running
* KAFKA producer running

This test will require manual verification of certain events and telemetry in the summit EFD.
Also manual verification of appropriate temperatures for each actuator.

In [2]:
from lsst.ts import salobj
from lsst.ts.idl.enums import MTHexapod
import logging
import asyncio
import os
import yaml

import numpy as np
from matplotlib import pyplot as plt
from astropy.time import Time
from datetime import datetime, timedelta
import pandas as pd
import astropy.units as u

from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5

from lsst_efd_client import EfdClient

#from lsst.ts.idl.enums.MTHexapod import ApplicationStatus
from lsst.ts.idl.enums.MTRotator import ApplicationStatus

#Auxillary telescope pointing component
from lsst.ts.idl.enums import ATPtg

from hexaTools import *

ModuleNotFoundError: No module named 'hexaTools'

In [ ]:
print(os.environ["OSPL_URI"])
print(os.environ["LSST_DDS_PARTITION_PREFIX"])

In [ ]:
#This step should fail at NCSA since there is only one domain
print(os.environ["LSST_DDS_DOMAIN_ID"])

In [ ]:
print(os.environ["LSST_DDS_INTERFACE"])

The following block sets the necessary environment variables for setting up the DDS/SAL communication

In [ ]:
#Wait for history data [sec] default value is 60
#os.environ["LSST_DDS_DOMAIN"] = 'citest'
#os.environ["LSST_DDS_HISTORYSYNC"] = "10"

In [ ]:
STD_WAIT = 39 # This is the amount of time to wait in between a move just to make sure that the actuators are not overheating

In [ ]:
#STD_WAIT = 70 # This is the amount of time to wait in between long moves just to make sure that the actuators are not overheating

In [ ]:
#STD_WAIT = 120 # This is the amount of time to wait in between very long moves to make sure that the actuators are not overheating

In [ ]:
log = logging.getLogger(__name__) # This sets up the logger for the test.
#logging.setLevel(20)
logging.basicConfig(level=logging.INFO)

In [ ]:
start_time = datetime.now()
test_message = "Camera Hexapod Integration Test"
#What means index=1 here?
#script = salobj.Controller("Script", index=1)
script = salobj.Controller("Script", index=42658886)
await asyncio.sleep(10) #wait 10 second may help with DDS problems; closing all other kernels may help too
print(f'time to start is {datetime.now() - start_time} [s]')

In [ ]:
await script.start_task

In [ ]:
#Connect to the ESS CSC
ess = salobj.Remote(script.domain, "ESS")

In [ ]:
await ess.start_task
try:
    temp = await ess.tel_temperature8Ch.next(flush=True, timeout=5)
except asyncio.exceptions.TimeoutError:
    print('Are the temperature sensors up??')

In [4]:
#Status/Error codes seen during the testing (especially during unplugging and re-plugging cables)

print("Code:20",ApplicationStatus(20))
print("Code:512",ApplicationStatus(512))

print("Code:4100",ApplicationStatus(4100))

print("Code:4116",ApplicationStatus(4116))
print("Code:4124", ApplicationStatus(4124))


print("Code:5126",ApplicationStatus(5126))
print("Code:5124",ApplicationStatus(5124))

print("Code:5140",ApplicationStatus(5140))
print("Code:5142",ApplicationStatus(5142))

print("Code:5172",ApplicationStatus(5172))
print("Code:5174",ApplicationStatus(5174))


print("Code:20484",ApplicationStatus(20484))

print("Code:20580",ApplicationStatus(20580))
print("Code:20584",ApplicationStatus(20584))


print("Code:21508",ApplicationStatus(21508))
print("Code:21516",ApplicationStatus(21516))

print("Code:21524",ApplicationStatus(21524))

print("Code:21540",ApplicationStatus(21540))
print("Code:21556",ApplicationStatus(21556))
print("Code:21580",ApplicationStatus(21580))

print("Code:29701",ApplicationStatus(29701))
print("Code:28676",ApplicationStatus(28676))
print("Code:28692",ApplicationStatus(28692))
print("Code:29700",ApplicationStatus(29700))

Code:20 ApplicationStatus.16|EUI_CONNECTED
Code:512 ApplicationStatus.ETHERCAT_PROBLEM
Code:4100 ApplicationStatus.DDS_CONNECTED|EUI_CONNECTED
Code:4116 ApplicationStatus.DDS_CONNECTED|16|EUI_CONNECTED
Code:4124 ApplicationStatus.DDS_CONNECTED|16|8|EUI_CONNECTED
Code:5126 ApplicationStatus.DDS_CONNECTED|DDS_COMMAND_SOURCE|EUI_CONNECTED|2
Code:5124 ApplicationStatus.DDS_CONNECTED|DDS_COMMAND_SOURCE|EUI_CONNECTED
Code:5140 ApplicationStatus.DDS_CONNECTED|DDS_COMMAND_SOURCE|16|EUI_CONNECTED
Code:5142 ApplicationStatus.DDS_CONNECTED|DDS_COMMAND_SOURCE|16|EUI_CONNECTED|2
Code:5172 ApplicationStatus.DDS_CONNECTED|DDS_COMMAND_SOURCE|COMMAND_REJECTED|16|EUI_CONNECTED
Code:5174 ApplicationStatus.DDS_CONNECTED|DDS_COMMAND_SOURCE|COMMAND_REJECTED|16|EUI_CONNECTED|2
Code:20484 ApplicationStatus.SIMULINK_FAULT|DDS_CONNECTED|EUI_CONNECTED
Code:20584 ApplicationStatus.SIMULINK_FAULT|DDS_CONNECTED|SAFETY_INTERLOCK|COMMAND_REJECTED|8
Code:20580 ApplicationStatus.SIMULINK_FAULT|DDS_CONNECTED|SAFETY_INTE

This is how you start the remote for the CSC.

In [ ]:
# create the remote
#index=1 is the camera hexapod!! index=2 is the M2 hexapod!
csc_index = 1
hexapod_csc = salobj.Remote(name="MTHexapod", domain=script.domain, index=csc_index)

In [ ]:
await hexapod_csc.start_task

In [ ]:
data = hexapod_csc.evt_softwareVersions.get()
print(data.cscVersion)

This is to verify that the CSC is running

In [ ]:
await hexapod_csc.evt_heartbeat.next(flush=True, timeout=5)

General command to transition the CSC and clear errors:

In [ ]:
await salobj.set_summary_state(hexapod_csc, salobj.State.OFFLINE)

In [ ]:
await salobj.set_summary_state(hexapod_csc, salobj.State.STANDBY)

In [ ]:
await salobj.set_summary_state(hexapod_csc, salobj.State.DISABLED)

In [ ]:
#This is to bring the state machine in the right starting state
await salobj.set_summary_state(hexapod_csc, salobj.State.ENABLED, settingsToApply="default")

The next four blocks bring the CSC to the Enabled state.

In [ ]:
await hexapod_csc.cmd_enterControl.set_start(timeout=10.) # Send the enterControl command notice the casing and the set_start method

In [ ]:
await hexapod_csc.cmd_exitControl.set_start(timeout=10.)

In [ ]:
await hexapod_csc.cmd_start.set_start(timeout=10.) 

In [ ]:
#This need the, settingsToApply="default" for the moment.
#await hexapod_csc.cmd_enable.set_start(timeout=10.)

In [ ]:
await hexapod_csc.cmd_clearError.set_start(timeout=10.) # This clears the error

In [ ]:
hexapod_csc.cmd_disable.set_start(timeout=10.)

In [ ]:
hexapod_csc.cmd_standby.set_start(timeout=10.)

In [ ]:
hexapod_csc.cmd_setLogLevel.set_start(timeout=10., Level=10)

In [ ]:
#Checking which hexapod we are controlling
print(hexapod_csc)
whichHex1 = await hexapod_csc.evt_configuration.aget(timeout=10.)
print("Hexapod hex", whichHex1.MTHexapodID)
print(hexapod_csc.salinfo.index)

These blocks can be used to move the hexapod and check the lookup table correction:

In [ ]:
#Check some Hexapod configurations 
hexConfig = await hexapod_csc.evt_configuration.aget(timeout=10.)
print("pivot at (%.0f, %.0f, %.0f) microns "%(hexConfig.pivotX, hexConfig.pivotY, hexConfig.pivotZ))
print("maxXY = ", hexConfig.maxXY, "microns, maxZ= ", hexConfig.maxZ, " microns")
print("maxUV = ", hexConfig.maxUV, "deg, maxW= ", hexConfig.maxW, " deg")

In [ ]:
camHexPosition0 = await hexapod_csc.tel_actuators.aget(timeout=10.)
print(camHexPosition0)

In [ ]:
print("camera Hexapod positions", camHexPosition0.MTHexapodID,  camHexPosition0.calibrated)

In [ ]:
#This cell makes sure that the data go to EFD
#may need to wait a few seconds before event shows up in EFD
#client = EfdClient('ncsa_teststand_efd')
client = EfdClient('summit_efd')

In [ ]:
end = Time(datetime.now())
start = end - timedelta(hours=3)
dfe = await client.select_time_series('lsst.sal.MTHexapod.logevent_summaryState', '*', start.tai, end.tai, csc_index)

In [ ]:
dfe

In [ ]:
def enabled_substate_callback(evt):
    """Print the enabled substate when event is received."""
    print(MTHexapod.EnabledSubstate(evt.enabledSubstate))

In [ ]:
def in_position_callback(evt):
    """Print the in position event when it is received."""
    print(evt.in_position)

This next block will make sure that the CSC is ready for the test

In [ ]:
connected = await hexapod_csc.evt_connected.aget(timeout=10.) # This is how you get an event/telemetry with await remote.type_name.aget() type being [evt,tel] and name being the name of the topic
#commandable = connected.command
telemetry_working = connected.telemetry

if not connected or not telemetry_working:
    raise Exception("Hexapod not connected or telemetry not being received.") 

In [ ]:
controller_state = await hexapod_csc.evt_controllerState.aget(timeout=10.)
state = controller_state.controllerState
offline_substate = controller_state.offlineSubstate
print(state)
#This is only for the real system. The simulater only has the AVAILABLE offline state 

'''  
if not state == salobj.State.OFFLINE and not offline_substate == MTHexapod.OfflineSubstate.AVAILABLE:
    raise Exception("Controller must be changed to Available Offline Substate.")
'''
commandable_by_dds = await hexapod_csc.evt_commandableByDDS.aget()
dds_state = commandable_by_dds.state
if not dds_state:
    raise Exception("Controller must in CommandableByDDS state.")

# thermal sensors
Check the chronograph manually for the temperature sensors to be below 19C, if not wait until they are all below 19C.

In [ ]:
dir(temp)

In [ ]:
[getattr(temp,'temperatureC%02d'%i) for i in range(1,8+1)]

In [ ]:
end = Time(datetime.now(), scale='tai')
start = end - timedelta(seconds=1000)
df = await client.select_time_series('lsst.sal.ESS.temperature8Ch', '*', start, end, csc_index)
fig, ax = plt.subplots(1,1, figsize=(15,4))
for i in range(1,8+1):
    plt.plot(getattr(df, 'temperatureC%02d'%i))
plt.grid()

In [ ]:
#Command used to stop the hexapod 
await hexapod_csc.cmd_stop.set_start()

In [ ]:
#Command used to stop the hexapod 
await hexapod_csc.cmd_abort.set_start()

# Test the Stop command
This test first move the Hexapod to zero and then allows the reaction to stop command after defined time.

In [ ]:
#This command is to set the Hexapod to zero position
now = datetime.now()
script.log.info(f"START -- {test_message} -- LVV-T1600 -- Move to xyz 0 -- Starting Time: {now} UTC")
await hexapod_csc.cmd_move.set_start(x=0,y=0,z=0, u=0,v=0,w=0,sync=True)

In [ ]:
# Test that stop interrupts move
# Start with the hexapod well away from z=500
# The test is a success if the hexapod remains close to its starting point.
# Start with delay = 0, then if that works try 0.1
# (the larger the delay the more the hexapod will move)
delay=0.1
await hexapod_csc.cmd_move.set_start(x=0, y=0, z=500, u=0, v=0, w=0,sync=True)
await asyncio.sleep(delay)
await hexapod_csc.cmd_stop.start()

In [ ]:
# Test that move interrupts move
# The test is a success if the hexapod ends up at the final position
# Start with delay = 0, then if that works try 0.1
# (the larger the delay the more the hexapod will move)
delay = 0.0
await hexapod_csc.cmd_move.set_start(x=0, y=0, z=450, u=0, v=0, w=0,sync=True)
await asyncio.sleep(delay)
await hexapod_csc.cmd_move.set_start(x=0, y=0, z=-450, u=0, v=0, w=0,sync=True)
await asyncio.sleep(delay)
await hexapod_csc.cmd_move.set_start(x=0, y=0, z=550, u=0, v=0, w=0,sync=True)
await asyncio.sleep(delay)
await hexapod_csc.cmd_move.set_start(x=0, y=0, z=-550, u=0, v=0, w=0,sync=True)

In [ ]:
script.log.info(f"TEST to see if annotations like this arrive in the EFD.")

In [ ]:
now = datetime.now()
script.log.info(f"START- {test_message} -- LVV-T1600 -move command test- Starting time: {now} UTC")

pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')

In [ ]:
hexapod_csc.evt_controllerState.callback = enabled_substate_callback                                
await hexapod_csc.cmd_move.set_start(x=500,y=-500,z=200, u=0.01,v=-0.015,w=0,sync=True)
await asyncio.sleep(15.)
pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')
now = datetime.now()
script.log.info(f"STOP- {test_message} -- LVV-T1600 -move command test- Finishing time: {now} UTC)

###### This command is to set the Hexapod to zero position
now = datetime.now()
script.log.info(f"START- {test_message} -- LVV-T1600 -- Move to Zero- Starting time: {now} UTC")
await hexapod_csc.cmd_move.set_start(x=0,y=0,z=0, u=0,v=0,w=0,sync=True)

In [ ]:
await asyncio.sleep(STD_WAIT)

In [ ]:
# thermal sensors
end = Time(datetime.now(), scale='tai')
start = end - timedelta(seconds=1000)
df = await client.select_time_series('lsst.sal.ESS.temperature8Ch', '*', start, end, csc_index)
fig, ax = plt.subplots(1,1, figsize=(15,4))
for i in range(1,8+1):
    plt.plot(getattr(df, 'temperatureC%02d'%i))
plt.grid()

This block moves the hexapod to a large position but stops it after waiting 3 seconds

In [ ]:
now = datetime.now()
script.log.info(f"START- {test_message} -- LVV-T1600 - stop command test- Starting time: {now} UTC")

pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')

In [ ]:
#Deviation for broken actuator number 6: z direciton changed from 5000 to -5000
await hexapod_csc.cmd_move.set_start(x=0,y=0,z=-5000,u=0,v=0,w=0,sync=True)

await asyncio.sleep(3)
await hexapod_csc.cmd_stop.set_start()

await asyncio.sleep(15.)

pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')


#cs = await hexapod_csc.evt_controllerState.aget()
#print(MTHexapod.EnabledSubstate(cs.enabledSubstate))
#if cs.enabledSubstate != MTHexapod.EnabledSubstate.CONTROLLED_STOPPING:
#     raise Exception(f"Controller's substate is not {MTHexapod.EnabledSubstate.CONTROLLED_STOPPING}. It is {MTHexapod.EnabledSubstate(cs.enabledSubstate)}")

# cs = await hexapod_csc.evt_controllerSubstate.aget()
# print(MTHexapod.EnabledSubstate(cs.enabledSubstate))
# if cs.enabledSubstate != MTHexapod.EnabledSubstate.STATIONARY:
#     raise Exception(f"Controller's substate is not {MTHexapod.EnabledSubstate.STATIONARY}. It is {MtHexapod.EnabledSubstate(cs.enabledSubstate)}")

script.log.info(f"STOP- {test_message} -- LVV-T1600 - stop command test- Finishing time: {now} UTC")

In [ ]:
#Step 15
await asyncio.sleep(STD_WAIT)

# thermal sensors
Check the chronograph manually for the temperature sensors to be below 19C, if not wait until they are all below 19C. Enter in the data at the 39 second mark

|Actuator 1 (C)|Actuator 2 (C) | Actuator 3 (C) | Actuator 4 (C) | Actuator 5 (C) | Actuator 6 (C)|
|--------------|---------------|----------------|----------------|----------------|---------------|
| 0 | 0 | 0 | 0 | 0 | 0 |

In [ ]:
#This command is to set the Hexapod to zero position
now = datetime.now()
script.log.info(f"START- {test_message} -- LVV-T1600 -- Move to Zero- Starting time: {now} UTC")
await hexapod_csc.cmd_move.set_start(x=0,y=0,z=0, u=0,v=0,w=0,sync=True)

In [ ]:
await asyncio.sleep(STD_WAIT)

CAVE: The setCompensationMode expects infrormation from the MTmount(elevation) and MTRotator(rotation).

To use the setCompensationMode at the summit the controller for MTMount needs to run and the actual MTRotator CSC needs to available at the summit.

To test this at NCSA the simulators must be stopped first.
Use the notebook MTMount-Controller the simulator, start the controller and reverse the steps after finishing testing.

For MTRotator the controller can not be used at NCSA. The simulator is always sending data even in OfflinePublishOnly state. Use the simulator at NCSA and the actual component at the summit

Start test for the lookup table compensation

In [ ]:
now = datetime.now()
script.log.info(f"START- {test_message} -- LVV-T1600 -compensationModeSet test- Starting time: {now} UTC")

Use test from BO to test the LUT compensation: https://github.com/lsst-ts/ts_notebooks/blob/tickets/DM-28409/bxin/ptg2hex/hex_diagnostics.ipynb

In [ ]:
#This command is obsolete
#await hexapod_csc.cmd_moveLUT.set_start(az=180,elev=60, temp=10)

In [ ]:
'''
await hexapod_csc.cmd_move.set_start(x=0,y=0,z=800,u=0,v=0,w=0,sync=True)
await asyncio.sleep(10)
pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')
'''

In [ ]:
#'''
#Switch CompensationMode on
await hexapod_csc.cmd_setCompensationMode.set_start(enable=True)

In [ ]:
lutMode = await hexapod_csc.evt_compensationMode.aget(timeout=10)
print("compsensation mode enabled?",lutMode.enabled, pd.to_datetime(lutMode.private_sndStamp, unit='s'))

In [ ]:
pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')
await asyncio.sleep(10.)
#'''

In [ ]:
#This command is to set the Hexapod to zero position
now = datetime.now()
script.log.info(f"START- {test_message} -- LVV-T1600 -- Move to Zero- Starting time: {now} UTC")
await hexapod_csc.cmd_move.set_start(x=0,y=0,z=0, u=0,v=0,w=0,sync=True)

In [ ]:
await printHexaPosition(hexapod_csc)
await printHexaUncompensatedAndCompensated(hexapod_csc)

In [ ]:
#move the hexapod Not allowed here! First leave compensation mode than move to new postion!!! Otherwise the you hexpod gets stuck in compensation  mode
#await hexapod_csc.cmd_move.set_start(x=0,y=0,z=800,u=0,v=0,w=0,sync=True)

#await asyncio.sleep(10.)

In [ ]:
'''
pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')

#Switch CompensationMode off 
await hexapod_csc.cmd_setCompensationMode.set_start(enable=False)
lutMode = await hexapod_csc.evt_compensationMode.aget(timeout=10)
print("compsensation mode enabled?",lutMode.enabled)
'''

In [ ]:
await hexapod_csc.cmd_setCompensationMode.set_start(enable=False)

In [ ]:
'''pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')
'''

In [ ]:
'''
posC = await hexapod_csc.evt_compensatedPosition.aget(timeout=10.)
print('Compensated position')
for i in 'xyzuvw':
    print('%10.2f '%getattr(posC, i), end= ' ')
print('\n')

posU = await hexapod_csc.evt_uncompensatedPosition.aget(timeout=10.)
print('Uncompensated position')
for i in 'xyzuvw':
    print('%10.2f '%getattr(posU, i), end= ' ')
print('\n')
'''

Analysis of the LUT correction

In [ ]:
'''LUTfile = '%s/notebooks/ts_config_mttcs/MTHexapod/v1/default.yaml'%(os.environ["HOME"])
print(LUTfile)
with open(LUTfile, 'r') as stream:
    aa = yaml.safe_load(stream)
if hexapod_csc.salinfo.index == 1:
    elevCoeff = aa['camera_config']['elevation_coeffs']
    tCoeff = aa['camera_config']['temperature_coeffs']
elif hexapod_csc.salinfo.index == 2:
    elevCoeff = aa['m2_config']['elevation_coeffs']
    tCoeff = aa['m2_config']['temperature_coeffs']
'''

In [ ]:
'''pred = []
elev=89.999
print('Predicted LUT compensation:')
for i in range(6):
    coeff = elevCoeff[i] #starts with C0
    mypoly = np.poly1d(coeff[::-1]) #poly1d needs C5 first
    pred.append(mypoly(elev))
    print('%10.2f '%pred[i], end= ' ')
print('\n')
print('Actual LUT compensation:')
for i in 'xyzuvw':
    print('%10.2f '%getattr(posC, i), end= ' ')
print('\n')
'''

In [ ]:
script.log.info(f"STOP- {test_message} -- LVV-T1600 -compensationModeSet test- Finishing time: {now} UTC"")

In [ ]:
#Step 22
await asyncio.sleep(STD_WAIT)
# thermal sensors

# thermal sensors
Check the chronograph manually for the temperature sensors to be below 19C, if not wait until they are all below 19C. Enter in the data at the 39 second mark

|Actuator 1 (C)|Actuator 2 (C) | Actuator 3 (C) | Actuator 4 (C) | Actuator 5 (C) | Actuator 6 (C)|
|--------------|---------------|----------------|----------------|----------------|---------------|
| 0 | 0 | 0 | 0 | 0 | 0 |

In [ ]:
now = datetime.now()
script.log.info(f"START- {test_message} -- LVV-T1600 Test Step 24 - Starting time: {now} UTC")

pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')

In [ ]:
await hexapod_csc.cmd_move.set_start(x=500,y=800,z=200,u=0,v=0,w=0,sync=True)
await asyncio.sleep(10.)

pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')

In [ ]:
await hexapod_csc.cmd_offset.set_start(x=0,y=0,z=500,u=0,v=0,w=0,sync=True)
await asyncio.sleep(10.)

pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')

script.log.info(f"STOP- {test_message} -- LVV-T1600 Test Step 24")


# cs = await MThexapod_csc.evt_controllerState.aget()
# if cs.enabledSubstate != MThexapod.SetEnabledSubstateParam.MOVE_POINT_TO_POINT:
#     raise Exception(f"Controller's substate is not {MThexapod.SetEnabledSubstateParam.MOVE_POINT_TO_POINT}. It is {MThexapod.SetEnabledSubstateParam(cs.enabledSubstate)}")
# cs = await MThexapod_csc.evt_controllerSubstate.aget()
# if cs.enabledSubstate != MThexapod.SetEnabledSubstateParam.STATIONARY:
#     raise Exception(f"Controller's substate is not {MThexapod.SetEnabledSubstateParam.STATIONARY}. It is {MThexapod.SetEnabledSubstateParam(cs.enabledSubstate)}")
# in_position = await MThexapod_csc.evt_inPosition.aget(timeout=10)
# if in_position.inPosition is False:
#     raise Exception("Controller not in position.")

In [ ]:
#This command is to set the Hexapod to zero position
now = datetime.now()
script.log.info(f"START -- {test_message} -- LVV-T1600 -- Move to Zero -- Starting time: {now} UTC")
await hexapod_csc.cmd_move.set_start(x=0,y=0,z=0, u=0,v=0,w=0,sync=True)

In [ ]:
#Step 27
await asyncio.sleep(STD_WAIT)

# Pivot tests 
Modifying the Pivot point, recovering the changes from the EFD. 

In [ ]:
now = datetime.now()
script.log.info(f"START- {test_message} -- LVV-T1600 Test Move - Pivot test - Starting time: {now} UTC")#Pivot test
#Test before pivot point test
await hexapod_csc.cmd_move.set_start(x=0,y=0,z=0,u=0.,v=0.1,w=0.0,sync=True)
await asyncio.sleep(10.)
pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('Hex position in X,Y,Z,U,V,W')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')

camHexPosition0 = await hexapod_csc.tel_actuators.aget(timeout=10.)
print('Hexapod calibrated actuator positions in um:')
for i in range(6):
    print('%10.2f '%camHexPosition0.calibrated[i], end= ' ')

In [ ]:
#Check some Hexapod configurations 
hexConfig = await hexapod_csc.evt_configuration.aget(timeout=10.)
print("pivot at (%.0f, %.0f, %.0f) microns "%(hexConfig.pivotX, hexConfig.pivotY, hexConfig.pivotZ))
print("maxXY = ", hexConfig.maxXY, "microns, maxZ= ", hexConfig.maxZ, " microns")
print("maxUV = ", hexConfig.maxUV, "deg, maxW= ", hexConfig.maxW, " deg")
pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('Hex position in X,Y,Z,U,V,W')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')

camHexPosition0 = await hexapod_csc.tel_actuators.aget(timeout=10.)
print('Hexapod calibrated actuator positions in um:')
for i in range(6):
    print('%10.2f '%camHexPosition0.calibrated[i], end= ' ')

In [ ]:
#move to testing position
now = datetime.now()
script.log.info(f"START- {test_message} -- LVV-T1600 Pivot test - Moving to testing position- Starting time: {now} UTC")
await hexapod_csc.cmd_move.set_start(x=2000,y=-3500,z=200,u=0.01,v=-0.05,w=0.002,sync=True)
await asyncio.sleep(10.)

pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('Hex position in X,Y,Z,U,V,W')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')

camHexPosition0 = await hexapod_csc.tel_actuators.aget(timeout=10.)
print('Hexapod calibrated actuator positions in um:')
for i in range(6):
    print('%10.2f '%camHexPosition0.calibrated[i], end= ' ')

In [ ]:
#Check some Hexapod configurations 
hexConfig = await hexapod_csc.evt_configuration.aget(timeout=10.)
print("pivot at (%.0f, %.0f, %.0f) microns "%(hexConfig.pivotX, hexConfig.pivotY, hexConfig.pivotZ))
print("maxXY = ", hexConfig.maxXY, "microns, maxZ= ", hexConfig.maxZ, " microns")
print("maxUV = ", hexConfig.maxUV, "deg, maxW= ", hexConfig.maxW, " deg")

In [ ]:
#Step 30 -- set a new Pivot point
now = datetime.now()
script.log.info(f"START- {test_message} -- LVV-T1600 Pivot test - Pivot point set to (0,0,0)- Starting time: {now} UTC")#Pivot test
await hexapod_csc.cmd_setPivot.set_start(x=0,y=0,z=0)
await asyncio.sleep(3.)

pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('Hex position in X,Y,Z,U,V,W')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')

camHexPosition0 = await hexapod_csc.tel_actuators.aget(timeout=10.)
print('Hexapod calibrated actuator positions in um:')
for i in range(6):
    print('%10.2f '%camHexPosition0.calibrated[i], end= ' ')
#await asyncio.sleep(10.)

Pivot point changed. Values changed in the EUI and in the EFD. Changes are shown in Chronograph.  Actuator Pos. Actual unchanged -- no movement

In [ ]:
#Check some Hexapod configurations 
hexConfig = await hexapod_csc.evt_configuration.aget(timeout=10.)
print("pivot at (%.0f, %.0f, %.0f) microns "%(hexConfig.pivotX, hexConfig.pivotY, hexConfig.pivotZ))
print("maxXY = ", hexConfig.maxXY, "microns, maxZ= ", hexConfig.maxZ, " microns")
print("maxUV = ", hexConfig.maxUV, "deg, maxW= ", hexConfig.maxW, " deg")

pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('Hex position in X,Y,Z,U,V,W')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')

camHexPosition0 = await hexapod_csc.tel_actuators.aget(timeout=10.)
print('Hexapod calibrated actuator positions in um:')
for i in range(6):
    print('%10.2f '%camHexPosition0.calibrated[i], end= ' ')

In [ ]:
#Step 31
script.log.info(f"START- {test_message} -- LVV-T1600 move to  (x=2000,y=-3500,z=200,u=0.01,v=-0.05,w=0.002,sync=True) again- Pivot test - Starting time: {now} UTC")#Pivot test
await hexapod_csc.cmd_move.set_start(x=2000,y=-3500,z=200,u=0.01,v=-0.05,w=0.002,sync=True)
await asyncio.sleep(10.)
pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('Hex position in X,Y,Z,U,V,W')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')

camHexPosition0 = await hexapod_csc.tel_actuators.aget(timeout=10.)
print('Hexapod calibrated actuator positions in um:')
for i in range(6):
    print('%10.2f '%camHexPosition0.calibrated[i], end= ' ')
    
#await asyncio.sleep(10.)

Position changed in the GUI and the EFD to the set value. Strud Pos Actual values changed.

In [ ]:
#Check some Hexapod configurations 
hexConfig = await hexapod_csc.evt_configuration.aget(timeout=10.)
print("pivot at (%.0f, %.0f, %.0f) microns "%(hexConfig.pivotX, hexConfig.pivotY, hexConfig.pivotZ))
print("maxXY = ", hexConfig.maxXY, "microns, maxZ= ", hexConfig.maxZ, " microns")
print("maxUV = ", hexConfig.maxUV, "deg, maxW= ", hexConfig.maxW, " deg")
pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('Hex position in X,Y,Z,U,V,W')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')

camHexPosition0 = await hexapod_csc.tel_actuators.aget(timeout=10.)
print('Hexapod calibrated actuator positions in um:')
for i in range(6):
    print('%10.2f '%camHexPosition0.calibrated[i], end= ' ')

In [ ]:
#Set the pivot point back to the original value (z=-2758400)

script.log.info(f"START- {test_message} -- LVV-T1600 Test Step 29 - Pivot set (0,0,-2758400)- Starting time: {now} UTC")#Pivot test
await hexapod_csc.cmd_setPivot.set_start(x=0,y=0,z=-2758400)
#await hexapod_csc.cmd_setPivot.set_start(x=0,y=0,z=-1938000)
#await hexapod_csc.cmd_setPivot.set_start(x=0,y=0,z=-703000)
await asyncio.sleep(3.)
pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')
script.log.info(f"STOP- {test_message} -- LVV-T1600 Test Step 29")

Values changed in teh EUI and in the EFD. Changes are shown in Chronograph. Strud. Pos. Actual unchanged -- no movement

In [ ]:
#Set the pivot point back to the original value (0.703m in um)
'''
script.log.info(f"START- {test_message} -- LVV-T1600 Test Step 29 - Pivot set (0,0,-1938000)- Starting time: {now} UTC")#Pivot test
await hexapod_csc.cmd_setPivot.set_start(x=0,y=0,z=-1938000)
#await hexapod_csc.cmd_setPivot.set_start(x=0,y=0,z=-2758400)
await asyncio.sleep(10.)
pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')

script.log.info(f"STOP- {test_message} -- LVV-T1600 Test Step 29")

'''

In [ ]:
#This command is to set the Hexapod to zero position
now = datetime.now()
script.log.info(f"START- {test_message} -- LVV-T1600 -- Move to Zero- Starting time: {now} UTC")
await hexapod_csc.cmd_move.set_start(x=0,y=0,z=0, u=0,v=0,w=0,sync=True)

In [ ]:
All moved back to zero.

# thermal sensors
Check the chronograph manually for the temperature sensors to be below 19C, if not wait until they are all below 19C. Enter in the data at the 39 second mark

|Actuator 1 (C)|Actuator 2 (C) | Actuator 3 (C) | Actuator 4 (C) | Actuator 5 (C) | Actuator 6 (C)|
|--------------|---------------|----------------|----------------|----------------|---------------|
| 0 | 0 | 0 | 0 | 0 | 0 |

In [ ]:
#Read test moves
test_moves = np.loadtxt('Test_Moves_D297.txt',comments='#')
test_moves = test_moves.T
#print (test_moves)

In [ ]:
now = datetime.now()
script.log.info(f"START- {test_message} -- LVV-T1600 Endurance Test - Starting time: {now} UTC")

for n in range(np.shape(test_moves)[1]):
    xyz_move = test_moves[:,n]
    await hexapod_csc.cmd_move.set_start(x=xyz_move[0],y=xyz_move[1],z=xyz_move[2],u=xyz_move[3],v=xyz_move[4],w=xyz_move[5],sync=True)
    #x=xyz_move[0]
    #y=xyz_move[1]
    #z=xyz_move[2]
    #u=xyz_move[3]
    #v=xyz_move[4]
    #w=xyz_move[5]
    #print(x,y,z,u,v,w)
    print ('Completed',n,'moves')
    script.log.info(f"RUNNING- {test_message} -- LVV-T1600 Endurance Test Step: {n}")
    await asyncio.sleep(STD_WAIT)

In [ ]:
# use this for the limit movement of actuator number 6
now = datetime.now()
script.log.info(f"START- {test_message} -- LVV-T1600 Endurance Test - Starting time: {now} UTC")
for i in range(40):
    await hexapod_csc.cmd_move.set_start(x=5660,y=0,z=7730,u=0,v=0.17,w=0,sync=True)
    print ('move 1 completed for the nth time:', i )
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=5660,y=0,z=7730,u=0,v=0.07,w=0,sync=True)
    print ('move 2 completed for the nth time:', i )
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=5660,y=0,z=7730,u=0.17,v=0,w=0,sync=True)
    print ('move 3 completed for the nth time:', i )
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=5660,y=0,z=7730,u=0.07,v=0,w=0,sync=True)
    print ('move 4 completed for the nth time:', i )
    await asyncio.sleep(STD_WAIT)

    await hexapod_csc.cmd_move.set_start(x=5660,y=0,z=-7730,u=0,v=0.17,w=0,sync=True)
    print ('move 5 completed for the nth time:', i )
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=5660,y=0,z=-7730,u=0,v=-0.17,w=0,sync=True)
    print ('move 6 completed for the nth time:', i )
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=5660,y=0,z=-7730,u=0.17,v=0,w=0,sync=True)
    print ('move 7 completed for the nth time:', i )
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=5660,y=0,z=-7730,u=-0.17,v=0,w=0,sync=True)
    print ('move 8 completed for the nth time:', i )
    await asyncio.sleep(STD_WAIT)

    await hexapod_csc.cmd_move.set_start(x=-5660,y=0,z=7730,u=0,v=0.17,w=0,sync=True)
    print ('move 9 completed for the nth time:', i )
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=-5660,y=0,z=7730,u=0,v=0.1,w=0,sync=True)
    print ('move 10 completed for the nth time:', i )
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=-5660,y=0,z=7730,u=0.17,v=0,w=0,sync=True)
    print ('move 11 completed for the nth time:', i )
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=-5660,y=0,z=7730,u=0.07,v=0,w=0,sync=True)
    print ('move 12 completed for the nth time:', i )
    await asyncio.sleep(STD_WAIT)

    await hexapod_csc.cmd_move.set_start(x=-5660,y=0,z=-7730,u=0,v=0.17,w=0,sync=True)
    print ('move 13 completed for the nth time:', i )
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=-5660,y=0,z=-7730,u=0,v=-0.17,w=0,sync=True)
    print ('move 14 completed for the nth time:', i )
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=-5660,y=0,z=-7730,u=0.17,v=0,w=0,sync=True)
    print ('move 15 completed for the nth time:', i )
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=-5660,y=0,z=-7730,u=-0.17,v=0,w=0,sync=True)
    print ('move 16 completed for the nth time:', i )
    await asyncio.sleep(STD_WAIT)


    #await hexapod_csc.cmd_move.set_start(x=0,y=5660,z=7730,u=0,v=0.17,w=0,sync=True)
    #print ('move 17 completed for the nth time:', i )
    #await asyncio.sleep(STD_WAIT)
    #await hexapod_csc.cmd_move.set_start(x=0,y=5660,z=7730,u=0,v=-0.17,w=0,sync=True)
    #print ('move 18 completed for the nth time:', i )
    #await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=5660,z=7730,u=0.17,v=0,w=0,sync=True)
    print ('move 19 completed for the nth time:', i ) 
    await asyncio.sleep(STD_WAIT)
    #await hexapod_csc.cmd_move.set_start(x=0,y=5660,z=7730,u=-0.17,v=0,w=0,sync=True)
    #print ('move 20 completed for the nth time:', i )
    #await asyncio.sleep(STD_WAIT)

    await hexapod_csc.cmd_move.set_start(x=0,y=5660,z=-7730,u=0,v=0.17,w=0,sync=True)
    print ('move 21 completed for the nth time:', i )
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=5660,z=-7730,u=0,v=-0.17,w=0,sync=True)
    print ('move 22 completed for the nth time:', i )
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=5660,z=-7730,u=0.17,v=0,w=0,sync=True)
    print ('move 23 completed for the nth time:', i )
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=5660,z=-7730,u=-0.17,v=0,w=0,sync=True)
    print ('move 24 completed for the nth time:', i )
    await asyncio.sleep(STD_WAIT)

    await hexapod_csc.cmd_move.set_start(x=0,y=-5660,z=7730,u=0,v=0.17,w=0,sync=True)
    print ('move 25 completed for the nth time:', i )
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=-5660,z=7730,u=0,v=-0.17,w=0,sync=True)
    print ('move 26 completed for the nth time:', i )
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=-5660,z=7730,u=0.17,v=0,w=0,sync=True)
    print ('move 27 completed for the nth time:', i )
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=-5660,z=7730,u=0.07,v=0,w=0,sync=True)
    print ('move 28 completed for the nth time:', i )
    await asyncio.sleep(STD_WAIT)

    await hexapod_csc.cmd_move.set_start(x=0,y=-5660,z=-7730,u=0,v=0.17,w=0,sync=True)
    print ('move 29 completed for the nth time:', i )
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=-5660,z=-7730,u=0,v=-0.17,w=0,sync=True)
    print ('move 30 completed for the nth time:', i )
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=-5660,z=-7730,u=0.17,v=0,w=0,sync=True)
    print ('move 31 completed for the nth time:', i )
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=-5660,z=-7730,u=-0.17,v=0,w=0,sync=True)
    print ('move 32 completed for the nth time:', i )
    await asyncio.sleep(STD_WAIT)
now = datetime.now()
script.log.info(f"STOPP- {test_message} -- LVV-T1600 Endurance Test - Stopping time: {now} UTC")

In [ ]:
#DO NOT USE until actuator number 6 is fixed!!
now = datetime.now()
script.log.info(f"START- {test_message} -- LVV-T1600 Endurance Test - Starting time: {now} UTC")
for i in range(100):
    '''
    await hexapod_csc.cmd_move.set_start(x=5660,y=0,z=7730,u=0,v=0.17,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=5660,y=0,z=7730,u=0,v=-0.17,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=5660,y=0,z=7730,u=0.17,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=5660,y=0,z=7730,u=-0.17,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)

    await hexapod_csc.cmd_move.set_start(x=5660,y=0,z=-7730,u=0,v=0.17,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=5660,y=0,z=-7730,u=0,v=-0.17,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=5660,y=0,z=-7730,u=0.17,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=5660,y=0,z=-7730,u=-0.17,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)

    await hexapod_csc.cmd_move.set_start(x=-5660,y=0,z=7730,u=0,v=0.17,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=-5660,y=0,z=7730,u=0,v=-0.17,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=-5660,y=0,z=7730,u=0.17,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=-5660,y=0,z=7730,u=-0.17,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)

    await hexapod_csc.cmd_move.set_start(x=-5660,y=0,z=-7730,u=0,v=0.17,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=-5660,y=0,z=-7730,u=0,v=-0.17,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=-5660,y=0,z=-7730,u=0.17,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=-5660,y=0,z=-7730,u=-0.17,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)


    await hexapod_csc.cmd_move.set_start(x=0,y=5660,z=7730,u=0,v=0.17,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=5660,z=7730,u=0,v=-0.17,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=5660,z=7730,u=0.17,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=5660,z=7730,u=-0.17,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)

    await hexapod_csc.cmd_move.set_start(x=0,y=5660,z=-7730,u=0,v=0.17,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=5660,z=-7730,u=0,v=-0.17,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=5660,z=-7730,u=0.17,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=5660,z=-7730,u=-0.17,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)

    await hexapod_csc.cmd_move.set_start(x=0,y=-5660,z=7730,u=0,v=0.17,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=-5660,z=7730,u=0,v=-0.17,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=-5660,z=7730,u=0.17,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=-5660,z=7730,u=-0.17,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)

    await hexapod_csc.cmd_move.set_start(x=0,y=-5660,z=-7730,u=0,v=0.17,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=-5660,z=-7730,u=0,v=-0.17,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=-5660,z=-7730,u=0.17,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=-5660,z=-7730,u=-0.17,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    '''
now = datetime.now()
script.log.info(f"STOPP- {test_message} -- LVV-T1600 Endurance Test - Stopping time: {now} UTC")

In [ ]:
await salobj.set_summary_state(hexapod_csc, salobj.State.STANDBY) # Transition the CSC to Standby state

In [ ]:
await domain.close() # Close the remote connection